In [1]:
import arcpy
import json
import os
import csv
from datetime import datetime

def get_export_type():
    """
    Prompt the user for the export type and validate the input.
    
    Returns:
    str: The validated export type ('csv', 'json', 'geojson').
    """
    while True:
        export_type = input("Enter export type (csv, json, geojson), or 'exit' to quit: ").strip().lower()
        if export_type in ['csv', 'json', 'geojson']:
            return export_type
        elif export_type == 'exit':
            print("Exiting the program.")
            return None
        else:
            print("Invalid export type. Please choose 'csv', 'json', or 'geojson'.")

def validate_feature_class(fc):
    """
    Validate the existence of the feature class.
    
    Parameters:
    fc (str): Path to the feature class.
    
    Returns:
    bool: True if valid, False otherwise.
    """
    if not arcpy.Exists(fc):
        print(f"Error: Feature class '{fc}' does not exist. Please provide a valid path.")
        return False
    return True

def export_feature_class(fc, export_type, output_dir):
    """
    Export the feature class to the specified format (csv, json, geojson).
    
    Parameters:
    fc (str): Path to the feature class.
    export_type (str): The export format ('csv', 'json', 'geojson').
    output_dir (str): Directory where the output file will be saved.
    """
    if not validate_feature_class(fc):
        return

    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = os.path.join(output_dir, f"feature_class_export_{current_time}.{export_type}")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created directory: {output_dir}")

    fields = [field.name for field in arcpy.ListFields(fc)] + ["SHAPE@"]

    features = []

    try:
        with arcpy.da.SearchCursor(fc, fields) as cursor:
            for row in cursor:
                feature = {field: value for field, value in zip(fields[:-1], row[:-1])}
                geometry = row[-1]
                if geometry:
                    geometry = geometry.projectAs(arcpy.SpatialReference(4326)).__geo_interface__
                feature["geometry"] = geometry
                features.append(feature)

        if not features:
            print("The feature class has no data to export.")
            return

        if export_type == 'geojson':
            geojson_output = {
                "type": "FeatureCollection",
                "features": [
                    {
                        "type": "Feature",
                        "geometry": feature["geometry"],
                        "properties": {key: feature[key] for key in feature if key != "geometry"}
                    }
                    for feature in features
                ]
            }
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(geojson_output, f, ensure_ascii=False, indent=4)
            print(f"Feature class exported to GeoJSON format: {output_file}")

        elif export_type == 'json':
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(features, f, ensure_ascii=False, indent=4)
            print(f"Feature class exported to JSON format: {output_file}")

        elif export_type == 'csv':
            with open(output_file, 'w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=features[0].keys())
                writer.writeheader()
                for feature in features:
                    writer.writerow(feature)
            print(f"Feature class exported to CSV format: {output_file}")

    except arcpy.ExecuteError as e:
        print(f"ArcPy error occurred: {e}")
    except Exception as e:
        print(f"General error occurred: {e}")

def run():
    """
    Main function to run the export process.
    """
    fc = input("Enter the path to the feature class: ").strip()
    output_dir = input("Enter the output directory: ").strip()

    while True:
        export_type = get_export_type()
        if export_type is None:
            break
        export_feature_class(fc, export_type, output_dir)

if __name__ == "__main__":
    run()


Created directory: D:\Exports
Feature class exported to CSV format: D:\Exports\feature_class_export_20241121_123424.csv
Exiting the program.
